In [ ]:
# ============================================================
# nb_silver_fx
# ------------------------------------------------------------
# Rôle :
#   - Transformer bronze_fx_raw -> silver_fx_rates
#   - Appliquer normalisation, typage, déduplication
#   - Ajouter traçabilité & audit
#   - Écrire la table Silver avec partitionnement mensuel
# ============================================================

# ------------------------------------------------------------
# 1) Import des utilitaires Silver
# ------------------------------------------------------------
# The command is not a standard IPython magic command. It is designed for use within Fabric notebooks only.
# %run ./nb_silver_utils

from pyspark.sql import functions as F

# ------------------------------------------------------------
# 2) Lecture de la source Bronze
# ------------------------------------------------------------
df_bronze = spark.table("bronze_fx_raw")

# ------------------------------------------------------------
# 3) Contrôles structurels (fail fast)
# ------------------------------------------------------------
assert_required_columns(
    df_bronze,
    [
        "currency",
        "base_currency",
        "currency_name",
        "rate_vs_usd",
        "date",
        "source_file",
        "ingestion_ts",
        "ingestion_date"
    ]
)

# ------------------------------------------------------------
# 4) Renommage des colonnes métier
# ------------------------------------------------------------
df = (
    df_bronze
    .withColumnRenamed("date", "fx_date")
    .withColumnRenamed("rate_vs_usd", "rate")
)


# ------------------------------------------------------------
# 5) Normalisations Silver (via utils)
# ------------------------------------------------------------

# 5.1 Dates FX (jour + mois)
df = add_fx_dates(df, date_col="fx_date")

# 5.2 Normalisation des codes devise (ISO)
df = normalize_currency_codes(df, cols=["currency", "base_currency"])

# 5.3 Taux FX (double -> decimal)
df = cast_rate(df, col="rate", precision=18, scale=8)

# ------------------------------------------------------------
# 6) Colonnes techniques & traçabilité
# ------------------------------------------------------------
df = add_tech_columns(
    df,
    source_file_col="source_file"
)

# ------------------------------------------------------------
# 7) Hash métier (stabilité & audit)
# ------------------------------------------------------------
df = add_record_hash(
    df,
    cols=[
        "currency",
        "base_currency",
        "fx_date",
        "rate"
    ]
)


# ------------------------------------------------------------
# 8) Déduplication Silver
# ------------------------------------------------------------
# Clé naturelle Bronze = (currency, date)
# On garde la forme robuste (base_currency, currency, fx_date)
df = deduplicate_latest(
    df,
    key_cols=["base_currency", "currency", "fx_date"],
    order_col="ingestion_ts"
)

# ------------------------------------------------------------
# 9) Projection finale du contrat Silver
# ------------------------------------------------------------
df = df.select(
    # --- Clés & temps ---
    "base_currency",
    "currency",
    "currency_name",
    "fx_date",
    "fx_month",

    # --- Mesure ---
    "rate",

    # --- Audit ---
    "source_file",
    "ingestion_date",
    "ingestion_ts",
    "record_hash"
)


# ------------------------------------------------------------
# 9bis) Contrôles gouvernance spécifiques FX (fail fast)
# ------------------------------------------------------------
bad_base = df.filter(
    F.col("base_currency").isNull() |
    (F.col("base_currency") != F.lit("EUR"))
)

if bad_base.limit(1).count() > 0:
    raise ValueError(
        "Invalid base_currency detected: expected EUR (EUR-based FX dataset)."
    )


# ------------------------------------------------------------
# 10) Écriture Silver gouvernée (partition mensuelle)
# ------------------------------------------------------------
write_silver_fx_rates(
    df,
    table_name="silver_fx_rates",
    mode="overwrite"
)

# ------------------------------------------------------------
# 11) Fin du notebook
# ------------------------------------------------------------
print("silver_fx_rates successfully written.")

